# GM Agent Logic - The Decision Maker

This notebook acts as the "Brain" of the GM Agent. It separates the **decision logic** from the pure **prediction logic**.

## Workflow
1. **State Observation**: Load current Team Roster & Cap Space.
2. **Candidate Evaluation**: Load Free Agents / Draft Class.
3. **Simulation Loop**:
    *   For each Candidate:
        *   Predict Player Performance (using `Player_Model_QB`, etc.) *in the context of MY team*.
        *   Update Temp Roster.
        *   Predict Team Wins (using `Team_Prediction`).
        *   Calculate ROI ($ cost per Win Added).
4. **Recommendation**: Rank candidates by ROI.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Mock Imports for now - In production these would load saved .keras models
print("Loading Predictive Models...")
# player_model = tf.keras.models.load_model('qb_transformer_model.keras')
# team_model = tf.keras.models.load_model('team_prediction_model.keras')

## Define Agent Tools

In [ ]:
def get_roster_context(team_id):
    # Fetch current O-Line Grade, WCS, etc.
    # Returns: Dict or Vector
    return {
        "OL_PassBlock": 75.0,
        "WR_Avg_Grade": 70.0,
        "Scheme": "WestCoast"
    }

def simulate_signing(player, team_context, salary_ask):
    # 1. Predict Player Performance in new context
    # predicted_grade = player_model.predict(player + team_context)
    predicted_grade = 80.0 # Mock
    
    # 2. Update Team Aggregates
    # new_team_stats = update_aggregates(team_context, predicted_grade)
    
    # 3. Predict Wins
    # wins = team_model.predict(new_team_stats)
    wins_base = 8.5
    wins_new = 9.2
    
    wins_added = wins_new - wins_base
    cost_per_win = salary_ask / wins_added if wins_added > 0 else float('inf')
    
    return {
        "Player": player['Name'],
        "Projected_Grade": predicted_grade,
        "Wins_Added": wins_added,
        "Cost_Per_Win": cost_per_win
    }

## Run Simulation

In [ ]:
# Example Free Agents
free_agents = [
    {"Name": "Kirk Cousins", "Position": "QB", "Salary": 35000000, "Stats": { ... }},
    {"Name": "Baker Mayfield", "Position": "QB", "Salary": 25000000, "Stats": { ... }}
]

my_team_context = get_roster_context("MIN")

results = []
for fa in free_agents:
    res = simulate_signing(fa, my_team_context, fa['Salary'])
    results.append(res)

# Decision Matrix
df_results = pd.DataFrame(results).sort_values(by='Cost_Per_Win')
print("GM Agent Recommendations:")
print(df_results)